In [1]:
import pandas as pd
import numpy as np
import polars as pl
from m3_model_params import lgb_params_1
from m4_feats_polars import *
from m5_sb_models import *
import warnings
warnings.filterwarnings("ignore")

data_path     = 'kaggle/input/linking-writing-processes-to-writing-quality/'
train_logs    = pl.scan_csv(f'{data_path}/train_logs.csv')
test_logs    = pl.scan_csv(f'{data_path}/test_logs.csv')
train_scores = pl.scan_csv(f'{data_path}/train_scores.csv')

In [2]:
# bad: sent_by_par, par_words, word_pauses_basic,

# neutral: cursor_pos_acceleration, countvectorize_two_one (slightly better than cursor_pos_acc + 4),  sent_timings, train_remove_word_pauses

# good: full sentences, full paragraphs, down_events, one_grams, create_pauses -  r-burst, nunique, words feats

# ratios

# 1. Pause time vs words Time
# 2. Number of pauses vs number of words
# 3. # events input / # number of events remove/cut
# 4. # events input / # events non-production


In [3]:
# Best Feature Set: ['train_essay_sentences.pkl', 'train_create_pauses.pkl', 
# 'train_vector_one_gram.pkl', 'train_essay_paragraphs.pkl', 
# 'train_categorical_nunique.pkl', 'train_word_pauses.pkl', 
# 'train_events_counts_rate_of_change.pkl', 'train_word_counts_rate_of_change.pkl', 
# 'train_r_burst_feats.pkl', 'train_vector_two_gram.pkl', 'train_events_counts_acceleration.pkl']

# Best_Feature_Set = ['train_down_events_counts.pkl', 'train_vector_one_gram.pkl', 
# 'train_create_pauses.pkl', 'train_sentences_per_paragraph.pkl', 
# 'train_add_word_pauses_basic.pkl', 'train_cursor_pos_acceleration.pkl', 
# 'train_remove_word_pauses_adv.pkl', 'train_paragraph_length.pkl', 
# 'train_categorical_nunique.pkl', 'train_paragraph_words.pkl', 
# 'train_events_counts_time_based.pkl', 'train_vector_two_gram.pkl', 
# 'train_sentences_words.pkl', 'train_add_word_pauses_adv.pkl']

# Best_Feature_Set = ['train_down_events_counts_one.pkl', 'train_essay_par_length.pkl', 
# 'train_create_pauses.pkl', 'train_countvectorize_one_two.pkl', 
# 'train_down_events_counts_two.pkl', 'train_cursor_pos_acceleration_basic.pkl', 
# 'train_essay_sents_per_par_basic.pkl', 'train_countvectorize_two_one.pkl', 
# 'train_remove_word_pauses_adv.pkl', 'train_add_word_pauses_adv.pkl', 
# 'train_word_count_acceleration_adv.pkl']


# Best Feature Set: ['train_down_events_counts.pkl', 'train_vector_one_gram.pkl', 
# 'train_create_pauses.pkl', 'train_essay_paragraphs.pkl', 
# 'train_cursor_pos_acceleration.pkl', 'train_sentences_per_paragraph.pkl', 
# 'train_remove_word_pauses.pkl', 'train_vector_two_gram.pkl', 
# 'train_p_burst_feats.pkl', 'train_r_burst_feats.pkl']


# Best Feature Set: ['train_down_events_counts.pkl', 'train_vector_one_gram.pkl', 
#                    'train_create_pauses.pkl', 'train_sentences_per_paragraph.pkl', 
#                    'train_essay_paragraphs.pkl', 'train_cursor_pos_acceleration.pkl', 
#                    'train_p_burst_feats.pkl', 'train_remove_word_pauses.pkl', 
#                    'train_vector_two_gram.pkl', 'train_remove_words_time_spent.pkl', 
#                    'train_product_to_keys.pkl']


# Best Feature Set: ['train_down_events_counts.pkl', 'train_vector_one_gram.pkl', 
# 'train_create_pauses.pkl', 'train_essay_paragraphs.pkl', 
# 'train_word_count_acceleration.pkl', 'train_remove_words_time_spent.pkl']

# Best Feature Set SVR: ['train_essay_sentences.pkl', 'train_create_pauses.pkl', 
# 'train_essay_paragraphs.pkl', 'train_word_wait_10.pkl', 
# 'train_categorical_nunique.pkl', 'train_remove_words_time_spent.pkl', 
# 'train_essay_words.pkl', 'train_word_wait_1.pkl', 
# 'train_add_word_pauses.pkl', 'train_word_wait_25.pkl']

## UNFINISHED XGBOOST
# ['train_down_events_counts.pkl', 'train_vector_one_gram.pkl', 
# 'train_create_pauses.pkl', 'train_essay_paragraphs.pkl', 
# 'train_essay_sentences.pkl', 'train_word_counts_rate_of_change.pkl', 
# 'train_vector_two_gram.pkl', 'train_text_changes_counts.pkl', 
# 'train_remove_word_pauses.pkl']

# Best Feature Set: ['train_down_events_counts.pkl', 'train_vector_one_gram.pkl', 
#                    'train_create_pauses.pkl', 'train_essay_paragraphs.pkl', 
#                    'train_activity_time_on_down_time.pkl', 'train_remove_word_pauses.pkl', 
#                    'train_cursor_pos_acceleration.pkl']


# cat <150

# lgbm 450

# xgb 225

In [4]:
# PANDAS FEATS
train_essays          = get_essay_df(train_logs.collect().to_pandas())
test_essays           = get_essay_df(test_logs.collect().to_pandas())

tr_down_events_counts, ts_down_events_counts = down_events_counts(train_logs, test_logs)
tr_vect_one, ts_vect_one = countvectorize_one_one(train_essays, test_essays)
tr_pauses, ts_pauses = create_pauses(train_logs, test_logs)
tr_word_c_acc, ts_word_c_acc = word_count_acceleration(train_logs, test_logs)
tr_rem_words_time_spent, ts_rem_words_time_spent = remove_words_time_spent(train_logs, test_logs)
tr_vect_two, ts_vect_two = countvectorize_two_one(train_essays, test_essays)
tr_cursor_pos_acc, ts_cursor_pos_acc = cursor_pos_acceleration(train_logs, test_logs)
tr_r_burst, ts_r_burst = r_burst_feats(train_logs, test_logs)
tr_nuni, ts_nuni = categorical_nunique(train_logs, test_logs)
tr_remove_pause, ts_remove_pause = remove_word_pauses(train_logs, test_logs)
tr_word_wait, ts_word_wait = word_wait_shift(train_logs, test_logs, 1)
tr_e_counts_roc, ts_e_counts_roc = events_counts_rate_of_change(train_logs, test_logs, time_agg=3)

# tr_wc_roc, ts_wc_roc = word_counts_rate_of_change(train_logs, test_logs)

train_feats = tr_down_events_counts.join(tr_vect_one, on='id', how='left')
train_feats = train_feats.join(tr_pauses, on='id', how='left')
train_feats = train_feats.join(tr_word_c_acc, on='id', how='left')
train_feats = train_feats.join(tr_rem_words_time_spent, on='id', how='left')
train_feats = train_feats.join(tr_vect_two, on='id', how='left')
train_feats = train_feats.join(tr_r_burst, on='id', how='left')
train_feats = train_feats.join(tr_cursor_pos_acc, on='id', how='left')
train_feats = train_feats.join(tr_nuni, on='id', how='left')
train_feats = train_feats.join(tr_remove_pause, on='id', how='left')
train_feats = train_feats.join(tr_word_wait, on='id', how='left')
train_feats = train_feats.join(tr_e_counts_roc, on='id', how='left')
# train_feats = train_feats.join(tr_wc_roc, on='id', how='left')


test_feats = ts_down_events_counts.join(ts_vect_one, on='id', how='left')
test_feats = test_feats.join(ts_pauses, on='id', how='left')
test_feats = test_feats.join(ts_word_c_acc, on='id', how='left')
test_feats = test_feats.join(ts_rem_words_time_spent, on='id', how='left')
test_feats = test_feats.join(ts_vect_two, on='id', how='left')
test_feats = test_feats.join(ts_r_burst, on='id', how='left')
test_feats = test_feats.join(ts_cursor_pos_acc, on='id', how='left')
test_feats = test_feats.join(ts_nuni, on='id', how='left')
test_feats = test_feats.join(ts_remove_pause, on='id', how='left')
test_feats = test_feats.join(ts_word_wait, on='id', how='left')
test_feats = test_feats.join(ts_e_counts_roc, on='id', how='left')
# test_feats = test_feats.join(ts_wc_roc, on='id', how='left')

train_logs = train_logs.collect().to_pandas()
test_logs = test_logs.collect().to_pandas()
train_scores = train_scores.collect().to_pandas()
train_feats = train_feats.collect().to_pandas()
test_feats = test_feats.collect().to_pandas()

train_feats           = train_feats.merge(parag_feats(train_essays), on='id', how='left')
test_feats            = test_feats.merge(parag_feats(test_essays), on='id', how='left')
train_feats           = train_feats.merge(sent_feats(train_essays), on='id', how='left')
test_feats            = test_feats.merge(sent_feats(test_essays), on='id', how='left')
train_feats           = train_feats.merge(word_feats(train_essays), on='id', how='left')
test_feats            = test_feats.merge(word_feats(test_essays), on='id', how='left')

train_feats           = train_feats.merge(train_scores, on=['id'], how='left')
print(f'train feats shape {train_feats.shape}')

< Events counts features >
< Count vectorize one-grams >
< Idle time features >
< word count acceleration >
< remove_words_time_spent >
< Count vectorize bi-grams >
< cursor position acceleration >
< R-burst features >
< Categorical # unique values features >
< removed words pauses basic
< word_wait_shift >
< event_id rate of change >
< Essays paragraphs feats >
< Essays paragraphs feats >
< Essays sentences feats >
< Essays sentences feats >
< Essays word feats >
< Essays word feats >
train feats shape (2471, 185)


In [5]:
from m5_sb_models import lgb_pipeline, xgb_pipeline
shuffle_preds = []

for est in [250,300,350,400,450,500]:

    xgb_params = {
        'alpha': 1,
        'colsample_bytree': 0.8,
        'gamma': 1.5,
        'learning_rate': 0.05,
        'max_depth': 4,
        'min_child_weight': 10,
        'subsample': 0.8,
        'n_estimators': est 
        }
    
    for i in range(8):
        train_feats = train_feats.sample(frac=1).reset_index(drop=True)
        test_preds, oof_preds, rmse, model = xgb_pipeline(train_feats, test_feats, xgb_params)
        shuffle_preds.append(rmse)
    print(f'Estimators {est}: {np.mean(shuffle_preds)}')

Estimators 250: 0.6041376201654391


In [ ]:
from m5_sb_models import catboost_pipeline

shuffle_preds = []
for est in [50,75,100,125,150,175,200,225,250,300,350,400,450,500]:

    catboost_params = {
        'iterations': est, 
        'learning_rate': 0.1, 
        'depth': 6, 
        'loss_function': 'RMSE', 
        'od_wait': 20, 
        'od_type': 'Iter', 
        'verbose': False, 
        'metric_period': 50, 
        'eval_metric': 'RMSE', 
        'bagging_temperature': 0.2
    }
        
    for i in range(15):
        train_feats = train_feats.sample(frac=1).reset_index(drop=True)
        test_preds, oof_preds, rmse, model = catboost_pipeline(train_feats, test_feats, catboost_params)
        shuffle_preds.append(rmse)
    print(f'Estimators {est}: {np.mean(shuffle_preds)}')

In [6]:
from m5_sb_models import lgb_pipeline, xgb_pipeline
shuffle_preds = []

for est in [250,300,350,400,450]:

    lgb_params_1 = {
        'boosting_type': 'gbdt', 
        'metric': 'rmse',
        'reg_alpha': 0.0031, 
        'reg_lambda': 0.001, 
        'colsample_bytree': 0.8,  
        'subsample_freq': 1,  
        'subsample': 0.75,  
        'learning_rate': 0.017, 
        'num_leaves': 19, 
        'min_child_samples': 46,
        'n_estimators': est,
        'verbosity': -1
        }
    
    for i in range(15):
        train_feats = train_feats.sample(frac=1).reset_index(drop=True)
        test_preds, oof_preds, rmse, model = lgb_pipeline(train_feats, test_feats, lgb_params_1)
        shuffle_preds.append(rmse)
    print(f'Estimators {est}: {np.mean(shuffle_preds)}')

Estimators 250: 0.6059676985712591
Estimators 300: 0.6055514276074799
Estimators 350: 0.6053147217238997
Estimators 400: 0.6051365824502887
Estimators 450: 0.6051214823348453


In [6]:
shuffle_preds = []

for i in range(15):
    train_feats = train_feats.sample(frac=1).reset_index(drop=True)
    test_preds, oof_preds, rmse, model = lgb_pipeline(train_feats, test_feats, lgb_params_1)
    shuffle_preds.append(rmse)

np.mean(shuffle_preds)

KeyboardInterrupt: 